# Instalar dependencias

In [1]:
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


# Instalar Theano

In [2]:
%pip install --upgrade --no-deps theano

Note: you may need to restart the kernel to use updated packages.


# Instalar Tensorflow y Keras


In [3]:
%pip install keras
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Redes Neuronales Artificales

# Cómo importar las librerías


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar el data set


In [5]:
dataset = pd.read_csv('./Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
# Ojo a la trampa de las variables ficticias: https://statologos.com/trampa-de-variables-ficticias/#:~:text=Trampa%20de%20variable%20ficticia%20%3A%20cuando,regresi%C3%B3n%20y%20los%20valores%20p.
# La eliminamos quitando la primera columna
X = X[:, 1:]

# Con la columna de sexo, que ahora es la 3, no hace falta hacer variables dummy, podemos directamente hacer el LabelEncoder (conversión de un género a 0 y otro a 1)
# Si lo pensamos, al hacer el proceso de convertir a variables dummy hay que eliminar una variable ficticia por lo que en las variables que solo pueden tener dos 
# categorías no tiene sentido hacer el proceso de variables dummy
labelencoder_X_2 = LabelEncoder()
X[:, 3] = labelencoder_X_2.fit_transform(X[:, 3])

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [8]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [9]:
# import keras
from keras.models import Sequential
from keras.layers import Dense

# Inicializar la RNA

In [10]:
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta


In [11]:
# Dense representa la sinapsis, conexión entre capas
#   - Units es el número de nodos que queremos en la primera capa oculta. Un consejo para establecer el número de nodos en la capa oculta es hacer la media entre el 
#     número de nodos de entrada y de salida. Aquí tenemos (11 + 1) / 2 = 6
#   - kernel_initializer se utiliza para inicializar los pesos. Se utiliza una distribución uniforme. Serán pequeños y cercanos a 0.
#   - input_dim es el número de nodos de entrada que se tienen. input shape es un tensor, hay que especificarle el tamaño de la muestra y la dimensión de cada dato (en este caso 11)
classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta

In [12]:
# En la segunda capa no hace falta input_dim ya que al venir de la anterior ya sabe que sería 6.
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

# Añadir la capa de salida

In [13]:
# - Al ser un resultado binario, se va o se queda del banco, solo habrá un nodo en la capa de salida.
#   Si se quisiese clasificar en tres categorías: cliente activo, pasivo, neutro en el banco...tendríamos 3 nodos. Ya no se usaría la sigmoide por no ser la más adecuada...
#   Deberíamos usar un escalón o un relu. Podríamos mantener la sigmoide pero probablemente tendríamos que usar softmax para que todas las probabilidades sumasen 1.
# - En la última capa utilizamos una función de activación sigmoide para tener la probabilidad de que la predicción pertenezca a una clase u otra.
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA

In [14]:
# Optimizer: es el algoritmo que se utiliza para encontrar el conjunto óptimo de pesos. Gradiente descendiente, gradiente descendiente estocástico, adam...
# loss (función de pérdidas): función que minimiza el error entre la predicción de la red y el valor real. ordinary least squares (minimización diferencias al cuadrado),
# Usamos la binary_crossentropy ya que es la más interesante para clasificación binaria.
# metrics: son las métricas que el sistema va a evaluar y las que va a intentar aumentar de una iteración a la siguiente.
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento

In [15]:
# batch_size: si recordamos la teoría, una vez que hemos hecho una pasada se actualiza el conjunto de pasos. Aquí definimos el tamaño del lote.
# epochs: definimos las veces que la red procesa todo el conjunto de datos para aprender.
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 790us/step - loss: 0.6136 - accuracy: 0.7955
Epoch 2/100
800/800 [==============================] - 1s 743us/step - loss: 0.5342 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 735us/step - loss: 0.5119 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 1s 737us/step - loss: 0.5070 - accuracy: 0.7960
Epoch 5/100
800/800 [==============================] - 1s 747us/step - loss: 0.5061 - accuracy: 0.7960
Epoch 6/100
800/800 [==============================] - 1s 741us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 7/100
800/800 [==============================] - 1s 745us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 8/100
800/800 [==============================] - 1s 756us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 9/100
800/800 [==============================] - 1s 732us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 10/100
800/800 [==============================] - 1s 718us/step - l

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [16]:
y_pred  = classifier.predict(X_test)
# Recordemos que las salidas son probabilidades, las convertimos a booleano
y_pred = (y_pred>0.5)

63/63 [==============================] - 0s 596us/step


# Elaborar una matriz de confusión

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

## Parte 4 - Evaluar, mejorar y Ajustar la RNA


### Instalamos dependencias

In [18]:
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.


### Evaluar la **RNA**


In [19]:
# Key-fold cross validation está en ambas librerías, scikit-learn y Keras. Utilizaremos el key-fold de scikit-learn en Keras
# Keras tiene wrappers de funciones de scikit-learn, permite utilizar funciones de scikit-learn en Keras. NOTA: al estar deprecados se recomienda migrar a los 
# wrappers de scikeras
# Importamos la función que hace el cross validation de scikit_learn
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

In [20]:
# Creamos función que devuelve el clasificador, la red neuronal
def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [21]:
# Creamos el objeto de clasificador de keras, le indicamos los parámetros para el entrenamiento
classifier = KerasClassifier(model = build_classifier, batch_size = 10, epochs = 100)
# Utilizamos la función cross_val_score: 
#   - estimator: es el que mide los errores, las precisiones. Será nuestro clasificador.
#   - X e y serán los datos de entrenamiento y pliegues. La función hará los folds.
#       Una técnica utilizada es tener un conjunto de entrenamiento, uno de test y luego un tercero de validación. Usamos una subparte del entrenamiento para validar
#       El de validación lo dejamos para el final del todo, para hacer predicciones y ver como va el clasificador.
#   - cv: número de folds. El estándar de pliegues es 10.
#   - n_jobs: número de procesadores que se van a utilizar para el entrenamiento. -1 utiliza el máximo.
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.1min finished


In [22]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

0.8397500000000001
0.0157599175124745


### Mejorar la RNA

#### Regularización de Dropout para evitar el *overfitting*

Overfitting y underfitting: https://www.youtube.com/watch?v=mQw2DCDsZVQ&ab_channel=JuanGabrielGomila
  - Overfitting o sobreajuste: el modelo funciona mucho mejor para predecir en el conjunto de entrenamiento que en el de testing. Una forma de verlo es cuando hay una gran varianza en el k-fold.
  - Una forma de evitarlo es añadir una capa de regularización por dropout. En cada una de las iteraciones esta capa desactiva aleatoriamente una o varias neuronas de la capa anterior para evitar que sean demasiado dependientes o que todas analicen y aprendan las mismas correlaciones y que una pueda sobreescribir a la otra. Evita que las neuronas se sobreespecialicen, haciendo que sean más independientes.
  - Lo más normal es aplicar esta capa de dropout después de una capa densa. Un consejo es aplicarlo después de cada capa, es cuando se tienen más oportunidades de evitar el sobreajuste.
  - Se añade una capa dropout que tenga baja probabilidad de desactivarse. Si se detecta que ese porcentaje es bajo se va incrementando. Esta probabilidad "rate" se interpreta como que si se pone 0.1 en un conjunto de entrada de 10 neuronas, a cada iteración se desactivará una neurona. Se recomienda empezar con 0.1 e ir subiendo de 0.1, en 0.1 hasta 0.5 como mucho.
  - Es mejor tener muchas capas de dropout después de cada capa densa que una capa de dropout con una fuerte rate.


In [10]:
from keras.layers import Dropout

def build_classifier_dropout():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dropout(rate = 0.1))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dropout(rate = 0.1))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

clasifier_dropout = KerasClassifier(model = build_classifier_dropout, batch_size = 10, epochs = 100)
accuracies_dropout = cross_val_score(estimator=clasifier_dropout, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

mean_dropout = accuracies_dropout.mean()
variance_dropout = accuracies_dropout.std()
print(mean_dropout)
print(variance_dropout)

KeyboardInterrupt: 

### Ajustar la *RNA*


In [9]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [10]:
# Añadimos un argumento a la función build_classifier por cada hiperparámetro que queremos optimizar
def build_classifier(optimizer = 'adam'):
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier


In [11]:
# El batch_size y el epochs ya no se indican, grid search buscará los mejores.
classifier = KerasClassifier(model = build_classifier)

In [12]:
# Definimos la rejilla de búsqueda con un diccionario con los parámetros y por cada uno de ellos un array con las distintas opciones para los parámetros.
parameters = {
    'batch_size' : [25,32], # Es una práctica muy común tomar potencia de 2
    'epochs' : [100, 500], 
    'optimizer' : ['adam', 'rmsprop'] # Podemos cambiar parámetros de la arquitectura de la RNA
}

In [13]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
288/288 [==============================] - 1s 728us/step - loss: 0.5554 - accuracy: 0.7961
Epoch 2/100
288/288 [==============================] - 0s 718us/step - loss: 0.4358 - accuracy: 0.7960
Epoch 3/100
288/288 [==============================] - 0s 711us/step - loss: 0.4309 - accuracy: 0.7960
Epoch 4/100
288/288 [==============================] - 0s 746us/step - loss: 0.4276 - accuracy: 0.7960
Epoch 5/100
288/288 [==============================] - 0s 770us/step - loss: 0.4250 - accuracy: 0.7960
Epoch 6/100
288/288 [==============================] - 0s 735us/step - loss: 0.4225 - accuracy: 0.7969
Epoch 7/100
288/288 [==============================] - 0s 718us/step - loss: 0.4207 - accuracy: 0.8146
Epoch 8/100
288/288 [==============================] - 0s 735us/step - loss: 0.4191 - accuracy: 0.8189
Epoch 9/100
288/288 [==============================] - 0s 721us/step - loss: 0.4184 - accuracy: 0.8224
Epoch 10/100
288/288 [==============================] - 0s 742us/step - l

288/288 [==============================] - 0s 718us/step - loss: 0.4018 - accuracy: 0.8354
Epoch 80/100
288/288 [==============================] - 0s 700us/step - loss: 0.4020 - accuracy: 0.8349
Epoch 81/100
288/288 [==============================] - 0s 711us/step - loss: 0.4021 - accuracy: 0.8349
Epoch 82/100
288/288 [==============================] - 0s 711us/step - loss: 0.4014 - accuracy: 0.8357
Epoch 83/100
288/288 [==============================] - 0s 697us/step - loss: 0.4017 - accuracy: 0.8340
Epoch 84/100
288/288 [==============================] - 0s 697us/step - loss: 0.4017 - accuracy: 0.8347
Epoch 85/100
288/288 [==============================] - 0s 711us/step - loss: 0.4018 - accuracy: 0.8363
Epoch 86/100
288/288 [==============================] - 0s 732us/step - loss: 0.4016 - accuracy: 0.8354
Epoch 87/100
288/288 [==============================] - 0s 693us/step - loss: 0.4018 - accuracy: 0.8351
Epoch 88/100
288/288 [==============================] - 0s 679us/step - loss:

288/288 [==============================] - 0s 704us/step - loss: 0.3910 - accuracy: 0.8392
Epoch 58/100
288/288 [==============================] - 0s 686us/step - loss: 0.3908 - accuracy: 0.8394
Epoch 59/100
288/288 [==============================] - 0s 700us/step - loss: 0.3905 - accuracy: 0.8397
Epoch 60/100
288/288 [==============================] - 0s 690us/step - loss: 0.3902 - accuracy: 0.8403
Epoch 61/100
288/288 [==============================] - 0s 700us/step - loss: 0.3901 - accuracy: 0.8382
Epoch 62/100
288/288 [==============================] - 0s 697us/step - loss: 0.3907 - accuracy: 0.8382
Epoch 63/100
288/288 [==============================] - 0s 732us/step - loss: 0.3903 - accuracy: 0.8385
Epoch 64/100
288/288 [==============================] - 0s 700us/step - loss: 0.3899 - accuracy: 0.8397
Epoch 65/100
288/288 [==============================] - 0s 767us/step - loss: 0.3901 - accuracy: 0.8394
Epoch 66/100
288/288 [==============================] - 0s 714us/step - loss:

288/288 [==============================] - 0s 700us/step - loss: 0.4043 - accuracy: 0.8351
Epoch 36/100
288/288 [==============================] - 0s 700us/step - loss: 0.4041 - accuracy: 0.8357
Epoch 37/100
288/288 [==============================] - 0s 672us/step - loss: 0.4039 - accuracy: 0.8347
Epoch 38/100
288/288 [==============================] - 0s 686us/step - loss: 0.4035 - accuracy: 0.8367
Epoch 39/100
288/288 [==============================] - 0s 690us/step - loss: 0.4044 - accuracy: 0.8356
Epoch 40/100
288/288 [==============================] - 0s 686us/step - loss: 0.4035 - accuracy: 0.8350
Epoch 41/100
288/288 [==============================] - 0s 700us/step - loss: 0.4035 - accuracy: 0.8357
Epoch 42/100
288/288 [==============================] - 0s 676us/step - loss: 0.4034 - accuracy: 0.8356
Epoch 43/100
288/288 [==============================] - 0s 676us/step - loss: 0.4035 - accuracy: 0.8354
Epoch 44/100
288/288 [==============================] - 0s 697us/step - loss:

288/288 [==============================] - 0s 735us/step - loss: 0.4146 - accuracy: 0.8285
Epoch 14/100
288/288 [==============================] - 0s 780us/step - loss: 0.4135 - accuracy: 0.8294
Epoch 15/100
288/288 [==============================] - 0s 742us/step - loss: 0.4125 - accuracy: 0.8310
Epoch 16/100
288/288 [==============================] - 0s 784us/step - loss: 0.4117 - accuracy: 0.8314
Epoch 17/100
288/288 [==============================] - 0s 749us/step - loss: 0.4108 - accuracy: 0.8313
Epoch 18/100
288/288 [==============================] - 0s 760us/step - loss: 0.4102 - accuracy: 0.8339
Epoch 19/100
288/288 [==============================] - 0s 693us/step - loss: 0.4095 - accuracy: 0.8324
Epoch 20/100
288/288 [==============================] - 0s 700us/step - loss: 0.4088 - accuracy: 0.8332
Epoch 21/100
288/288 [==============================] - 0s 700us/step - loss: 0.4087 - accuracy: 0.8332
Epoch 22/100
288/288 [==============================] - 0s 787us/step - loss:

...

320/320 [==============================] - 0s 674us/step - loss: 0.3355 - accuracy: 0.8622
Epoch 176/500
320/320 [==============================] - 0s 671us/step - loss: 0.3355 - accuracy: 0.8614
Epoch 177/500
320/320 [==============================] - 0s 671us/step - loss: 0.3343 - accuracy: 0.8633
Epoch 178/500
320/320 [==============================] - 0s 671us/step - loss: 0.3340 - accuracy: 0.8610
Epoch 179/500
320/320 [==============================] - 0s 674us/step - loss: 0.3351 - accuracy: 0.8620
Epoch 180/500
320/320 [==============================] - 0s 671us/step - loss: 0.3351 - accuracy: 0.8631
Epoch 181/500
320/320 [==============================] - 0s 668us/step - loss: 0.3352 - accuracy: 0.8622
Epoch 182/500
320/320 [==============================] - 0s 680us/step - loss: 0.3359 - accuracy: 0.8608
Epoch 183/500
320/320 [==============================] - 0s 677us/step - loss: 0.3340 - accuracy: 0.8620
Epoch 184/500
320/320 [==============================] - 0s 677us/ste

320/320 [==============================] - 0s 674us/step - loss: 0.3337 - accuracy: 0.8629
Epoch 254/500
320/320 [==============================] - 0s 671us/step - loss: 0.3340 - accuracy: 0.8644
Epoch 255/500
320/320 [==============================] - 0s 671us/step - loss: 0.3350 - accuracy: 0.8637
Epoch 256/500
320/320 [==============================] - 0s 674us/step - loss: 0.3333 - accuracy: 0.8630
Epoch 257/500
320/320 [==============================] - 0s 674us/step - loss: 0.3332 - accuracy: 0.8635
Epoch 258/500
320/320 [==============================] - 0s 677us/step - loss: 0.3338 - accuracy: 0.8634
Epoch 259/500
320/320 [==============================] - 0s 668us/step - loss: 0.3343 - accuracy: 0.8602
Epoch 260/500
320/320 [==============================] - 0s 668us/step - loss: 0.3340 - accuracy: 0.8636
Epoch 261/500
320/320 [==============================] - 0s 674us/step - loss: 0.3342 - accuracy: 0.8630
Epoch 262/500
320/320 [==============================] - 0s 677us/ste

320/320 [==============================] - 0s 680us/step - loss: 0.3326 - accuracy: 0.8620
Epoch 332/500
320/320 [==============================] - 0s 689us/step - loss: 0.3338 - accuracy: 0.8622
Epoch 333/500
320/320 [==============================] - 0s 674us/step - loss: 0.3335 - accuracy: 0.8634
Epoch 334/500
320/320 [==============================] - 0s 668us/step - loss: 0.3339 - accuracy: 0.8616
Epoch 335/500
320/320 [==============================] - 0s 671us/step - loss: 0.3339 - accuracy: 0.8644
Epoch 336/500
320/320 [==============================] - 0s 677us/step - loss: 0.3340 - accuracy: 0.8634
Epoch 337/500
320/320 [==============================] - 0s 674us/step - loss: 0.3332 - accuracy: 0.8618
Epoch 338/500
320/320 [==============================] - 0s 671us/step - loss: 0.3331 - accuracy: 0.8646
Epoch 339/500
320/320 [==============================] - 0s 668us/step - loss: 0.3346 - accuracy: 0.8635
Epoch 340/500
320/320 [==============================] - 0s 677us/ste

320/320 [==============================] - 0s 668us/step - loss: 0.3332 - accuracy: 0.8646
Epoch 410/500
320/320 [==============================] - 0s 677us/step - loss: 0.3331 - accuracy: 0.8640
Epoch 411/500
320/320 [==============================] - 0s 683us/step - loss: 0.3334 - accuracy: 0.8643
Epoch 412/500
320/320 [==============================] - 0s 671us/step - loss: 0.3326 - accuracy: 0.8616
Epoch 413/500
320/320 [==============================] - 0s 668us/step - loss: 0.3333 - accuracy: 0.8629
Epoch 414/500
320/320 [==============================] - 0s 677us/step - loss: 0.3336 - accuracy: 0.8624
Epoch 415/500
320/320 [==============================] - 0s 674us/step - loss: 0.3331 - accuracy: 0.8629
Epoch 416/500
320/320 [==============================] - 0s 672us/step - loss: 0.3334 - accuracy: 0.8654
Epoch 417/500
320/320 [==============================] - 0s 674us/step - loss: 0.3333 - accuracy: 0.8615
Epoch 418/500
320/320 [==============================] - 0s 671us/ste

320/320 [==============================] - 0s 677us/step - loss: 0.3330 - accuracy: 0.8640
Epoch 488/500
320/320 [==============================] - 0s 693us/step - loss: 0.3330 - accuracy: 0.8639
Epoch 489/500
320/320 [==============================] - 0s 737us/step - loss: 0.3322 - accuracy: 0.8624
Epoch 490/500
320/320 [==============================] - 0s 730us/step - loss: 0.3330 - accuracy: 0.8630
Epoch 491/500
320/320 [==============================] - 0s 677us/step - loss: 0.3333 - accuracy: 0.8636
Epoch 492/500
320/320 [==============================] - 0s 746us/step - loss: 0.3329 - accuracy: 0.8639
Epoch 493/500
320/320 [==============================] - 0s 705us/step - loss: 0.3328 - accuracy: 0.8637
Epoch 494/500
320/320 [==============================] - 0s 762us/step - loss: 0.3330 - accuracy: 0.8618
Epoch 495/500
320/320 [==============================] - 0s 674us/step - loss: 0.3329 - accuracy: 0.8652
Epoch 496/500
320/320 [==============================] - 0s 674us/ste

In [14]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'epochs': 500, 'optimizer': 'adam'}
0.85375
